# Skip-gram word2vec

In this notebook, I'll lead you through using TensorFlow to implement the word2vec algorithm using the skip-gram architecture. By implementing this, you'll learn about embedding words for use in natural language processing. This will come in handy when dealing with things like translations.

## Readings

Here are the resources I used to build this notebook. I suggest reading these either beforehand or while you're working on this material.

* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of word2vec from Chris McCormick 
* [First word2vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [NIPS paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for word2vec also from Mikolov et al.
* An [implementation of word2vec](http://www.thushv.com/natural_language_processing/word2vec-part-1-nlp-with-deep-learning-with-tensorflow-skip-gram/) from Thushan Ganegedara
* TensorFlow [word2vec tutorial](https://www.tensorflow.org/tutorials/word2vec)

## Word embeddings

When you're dealing with language and words, you end up with tens of thousands of classes to predict, one for each word. Trying to one-hot encode these words is massively inefficient, you'll have one element set to 1 and the other 50,000 set to 0. The word2vec algorithm finds much more efficient representations by finding vectors that represent the words. These vectors also contain semantic information about the words. Words that show up in similar contexts, such as "black", "white", and "red" will have vectors near each other. There are two architectures for implementing word2vec, CBOW (Continuous Bag-Of-Words) and Skip-gram.

<img src="assets/word2vec_architectures.png" width="500">

In this implementation, we'll be using the skip-gram architecture because it performs better than CBOW. Here, we pass in a word and try to predict the words surrounding it in the text. In this way, we can train the network to learn representations for words that show up in similar contexts.

First up, importing packages.

In [1]:
import time

import numpy as np
import tensorflow as tf

import utils

Load the [text8 dataset](http://mattmahoney.net/dc/textdata.html), a file of cleaned up Wikipedia articles from Matt Mahoney. The next cell will download the data set to the `data` folder. Then you can extract it and delete the archive file to save storage space.

In [2]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'data'
dataset_filename = 'text8.zip'
dataset_name = 'Text8 Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'http://mattmahoney.net/dc/text8.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)
        
with open('data/text8') as f:
    text = f.read()

## Preprocessing

Here I'm fixing up the text to make training easier. This comes from the `utils` module I wrote. The `preprocess` function coverts any punctuation into tokens, so a period is changed to ` <PERIOD> `. In this data set, there aren't any periods, but it will help in other NLP problems. I'm also removing all words that show up five or fewer times in the dataset. This will greatly reduce issues due to noise in the data and improve the quality of the vector representations. If you want to write your own functions for this stuff, go for it.

In [3]:
words = utils.preprocess(text)
print(words[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [4]:
print("Total words: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words: 16680599
Unique words: 63641


And here I'm creating dictionaries to covert words to integers and backwards, integers to words. The integers are assigned in descending frequency order, so the most frequent word ("the") is given the integer 0 and the next most frequent is 1 and so on. The words are converted to integers and stored in the list `int_words`.

In [5]:
vocab_to_int, int_to_vocab = utils.create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

## Subsampling

Words that show up often such as "the", "of", and "for" don't provide much context to the nearby words. If we discard some of them, we can remove some of the noise from our data and in return get faster training and better representations. This process is called subsampling by Mikolov. For each word $w_i$ in the training set, we'll discard it with probability given by 

$$ P(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}} $$

where $t$ is a threshold parameter and $f(w_i)$ is the frequency of word $w_i$ in the total dataset.

I'm going to leave this up to you as an exercise. This is more of a programming challenge, than about deep learning specifically. But, being able to prepare your data for your network is an important skill to have. Check out my solution to see how I did it.

> **Exercise:** Implement subsampling for the words in `int_words`. That is, go through `int_words` and discard each word given the probablility $P(w_i)$ shown above. Note that $P(w_i)$ is the probability that a word is discarded. Assign the subsampled data to `train_words`.

In [6]:
from collections import Counter
import random

## reference: https://www.quora.com/How-does-sub-sampling-of-frequent-words-work-in-the-context-of-Word2Vec
t = 1e-5
word_count = Counter(int_words)
word_len   = len(int_words)

frequency = {word: count/word_len for word,count in word_count.items()}
prob = {word: 1 - np.sqrt(t / frequency[word]) for word in word_count}

train_words = [word for word in int_words if prob[word]< random.random()]

## Making batches

Now that our data is in good shape, we need to get it into the proper form to pass it into our network. With the skip-gram architecture, for each word in the text, we want to grab all the words in a window around that word, with size $C$. 

From [Mikolov et al.](https://arxiv.org/pdf/1301.3781.pdf): 

"Since the more distant words are usually less related to the current word than those close to it, we give less weight to the distant words by sampling less from those words in our training examples... If we choose $C = 5$, for each training word we will select randomly a number $R$ in range $< 1; C >$, and then use $R$ words from history and $R$ words from the future of the current word as correct labels."

> **Exercise:** Implement a function `get_target` that receives a list of words, an index, and a window size, then returns a list of words in the window around the index. Make sure to use the algorithm described above, where you choose a random number of words from the window.

In [7]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    # Your code here
    R = np.random.randint(1, window_size+1) #integers from 1 to 5
    start = idx - R if (idx - R) >0 else 0
    stop  = idx + R
    
    #set to get rid of the duplicates
    target_words = set(words[start:idx] + words[idx+1:stop+1])
        
    
    return list(target_words)

Here's a function that returns batches for our network. The idea is that it grabs `batch_size` words from a words list. Then for each of those words, it gets the target words in the window. I haven't found a way to pass in a random number of target words and get it to work with the architecture, so I make one row per input-target pair. This is a generator function by the way, helps save memory.

In [8]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y
    

## Building the graph

From Chris McCormick's blog, we can see the general structure of our network.
![embedding_network](./assets/skip_gram_net_arch.png)

The input words are passed in as one-hot encoded vectors. This will go into a hidden layer of linear units, then into a softmax layer. We'll use the softmax layer to make a prediction like normal.

The idea here is to train the hidden layer weight matrix to find efficient representations for our words. This weight matrix is usually called the embedding matrix or embedding look-up table. We can discard the softmax layer becuase we don't really care about making predictions with this network. We just want the embedding matrix so we can use it in other networks we build from the dataset.

I'm going to have you build the graph in stages now. First off, creating the `inputs` and `labels` placeholders like normal.

> **Exercise:** Assign `inputs` and `labels` using `tf.placeholder`. We're going to be passing in integers, so set the data types to `tf.int32`. The batches we're passing in will have varying sizes, so set the batch sizes to [`None`]. To make things work later, you'll need to set the second dimension of `labels` to `None` or `1`.

In [9]:
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, [None], name='inputs')
    labels = tf.placeholder(tf.int32, [None, None], name='labels')

## Embedding



The embedding matrix has a size of the number of words by the number of neurons in the hidden layer. So, if you have 10,000 words and 300 hidden units, the matrix will have size $10,000 \times 300$. Remember that we're using one-hot encoded vectors for our inputs. When you do the matrix multiplication of the one-hot vector with the embedding matrix, you end up selecting only one row out of the entire matrix:

![one-hot matrix multiplication](assets/matrix_mult_w_one_hot.png)

You don't actually need to do the matrix multiplication, you just need to select the row in the embedding matrix that corresponds to the input word. Then, the embedding matrix becomes a lookup table, you're looking up a vector the size of the hidden layer that represents the input word.

<img src="assets/word2vec_weight_matrix_lookup_table.png" width=500>


> **Exercise:** Tensorflow provides a convenient function [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup) that does this lookup for us. You pass in the embedding matrix and a tensor of integers, then it returns rows in the matrix corresponding to those integers. Below, set the number of embedding features you'll use (200 is a good start), create the embedding matrix variable, and use [`tf.nn.embedding_lookup`](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup) to get the embedding tensors. For the embedding matrix, I suggest you initialize it with a uniform random numbers between -1 and 1 using [tf.random_uniform](https://www.tensorflow.org/api_docs/python/tf/random_uniform). This [TensorFlow tutorial](https://www.tensorflow.org/tutorials/word2vec) will help if you get stuck.

In [10]:
n_vocab = len(int_to_vocab)
n_embedding =  200
with train_graph.as_default():
    #initialized from -1 to 1
    embedding = tf.Variable(tf.random_uniform((n_vocab, n_embedding), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, inputs)

## Negative sampling



For every example we give the network, we train it using the output from the softmax layer. That means for each input, we're making very small changes to millions of weights even though we only have one true example. This makes training the network very inefficient. We can approximate the loss from the softmax layer by only updating a small subset of all the weights at once. We'll update the weights for the correct label, but only a small number of incorrect labels. This is called ["negative sampling"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). Tensorflow has a convenient function to do this, [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss).

> **Exercise:** Below, create weights and biases for the softmax layer. Then, use [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss) to calculate the loss. Be sure to read the documentation to figure out how it works.

In [11]:
# Number of negative labels to sample
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal((n_vocab, n_embedding), stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(n_vocab))
    
    # Calculate the loss using negative sampling
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, labels,
                                      embed, n_sampled,n_vocab)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

## Validation

This code is from Thushan Ganegedara's implementation. Here we're going to choose a few common words and few uncommon words. Then, we'll print out the closest words to them. It's a nice way to check that our embedding table is grouping together words with similar semantic meanings.

In [12]:
with train_graph.as_default():
    ## From Thushan Ganegedara's implementation
    valid_size = 16 # Random set of words to evaluate similarity on.
    valid_window = 100
    # pick 8 samples from (0,100) and (1000,1100) each ranges. lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000,1000+valid_window), valid_size//2))

    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keep_dims=True))
    normalized_embedding = embedding / norm
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [13]:
# If the checkpoints directory doesn't exist:
!mkdir checkpoints

## Training

Below is the code to train the network. Every 100 batches it reports the training loss. Every 1000 batches, it'll print out the validation words.

In [14]:
epochs = 10
batch_size = 1000
window_size = 10

with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())

    for e in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        for x, y in batches:
            
            feed = {inputs: x,
                    labels: np.array(y)[:, None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict=feed)
            
            loss += train_loss
            
            if iteration % 100 == 0: 
                end = time.time()
                print("Epoch {}/{}".format(e, epochs),
                      "Iteration: {}".format(iteration),
                      "Avg. Training loss: {:.4f}".format(loss/100),
                      "{:.4f} sec/batch".format((end-start)/100))
                loss = 0
                start = time.time()
            
            if iteration % 1000 == 0:
                ## From Thushan Ganegedara's implementation
                # note that this is expensive (~20% slowdown if computed every 500 steps)
                sim = similarity.eval()
                for i in range(valid_size):
                    valid_word = int_to_vocab[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log = 'Nearest to %s:' % valid_word
                    for k in range(top_k):
                        close_word = int_to_vocab[nearest[k]]
                        log = '%s %s,' % (log, close_word)
                    print(log)
            
            iteration += 1
    save_path = saver.save(sess, "checkpoints/text8.ckpt")
    embed_mat = sess.run(normalized_embedding)

Epoch 1/10 Iteration: 100 Avg. Training loss: 5.6444 0.2901 sec/batch
Epoch 1/10 Iteration: 200 Avg. Training loss: 5.6080 0.3057 sec/batch
Epoch 1/10 Iteration: 300 Avg. Training loss: 5.5163 0.3350 sec/batch
Epoch 1/10 Iteration: 400 Avg. Training loss: 5.6330 0.3560 sec/batch
Epoch 1/10 Iteration: 500 Avg. Training loss: 5.4891 0.3646 sec/batch
Epoch 1/10 Iteration: 600 Avg. Training loss: 5.5213 0.3947 sec/batch
Epoch 1/10 Iteration: 700 Avg. Training loss: 5.5486 0.3942 sec/batch
Epoch 1/10 Iteration: 800 Avg. Training loss: 5.5818 0.4058 sec/batch
Epoch 1/10 Iteration: 900 Avg. Training loss: 5.4776 0.4137 sec/batch
Epoch 1/10 Iteration: 1000 Avg. Training loss: 5.4415 0.4106 sec/batch
Nearest to they: shotcrete, ashur, otter, worshiped, assembly, nuevo, jeffries, zeroes,
Nearest to may: repertory, currently, mesothelioma, salter, hentai, mabinogion, undisturbed, mprp,
Nearest to states: luc, galway, illich, appeared, birch, godlike, tvs, owns,
Nearest to his: international, viol

Nearest to operating: exploits, sacked, under, hawke, setlist, exact, mohs, intercept,
Nearest to heavy: journal, ashland, osnabr, dynamic, disagreed, overviews, northeast, purchasing,
Nearest to bbc: undisclosed, sewing, singer, xvi, polling, bipartisan, equating, sidney,
Nearest to operations: abu, seleucids, ordinal, censure, payback, patagonian, amerindian, usable,
Nearest to mean: considerably, harmonization, swayed, archontology, modifications, unsolved, hoysala, mpla,
Nearest to institute: insidious, fieldwork, uno, crimes, whaler, five, pyongyang, clams,
Epoch 1/10 Iteration: 4100 Avg. Training loss: 4.6830 0.3748 sec/batch
Epoch 1/10 Iteration: 4200 Avg. Training loss: 4.7069 0.3877 sec/batch
Epoch 1/10 Iteration: 4300 Avg. Training loss: 4.5990 0.3801 sec/batch
Epoch 1/10 Iteration: 4400 Avg. Training loss: 4.6542 0.3808 sec/batch
Epoch 1/10 Iteration: 4500 Avg. Training loss: 4.6312 0.3813 sec/batch
Epoch 1/10 Iteration: 4600 Avg. Training loss: 4.6238 0.3742 sec/batch
Epoch

Nearest to his: languedoc, adultery, violation, bride, story, erupt, rose, aspire,
Nearest to that: against, prosecute, outcome, dreamlike, fulfilled, supported, voluntarily, lend,
Nearest to a: integrals, applicable, be, datapoint, ghent, erase, sociopolitical, pessimistic,
Nearest to with: winding, nighthawk, pivots, define, cabal, knees, resuscitation, shared,
Nearest to after: granting, commissioned, tipping, para, ingram, in, suicide, flushing,
Nearest to shows: hillis, beadwork, cavalier, advise, stop, rancho, personality, rubens,
Nearest to universe: subtitled, calculable, accelerating, elaborated, montpellier, recursive, genres, unmarried,
Nearest to operating: required, systems, unix, setlist, laptop, available, exploits, relay,
Nearest to heavy: journal, osnabr, ashland, manny, overviews, purchasing, grands, cleaved,
Nearest to bbc: sewing, singer, undisclosed, xvi, dixie, april, poached, chatham,
Nearest to operations: seleucids, censure, usable, cpe, abu, nessie, measuremen

Epoch 3/10 Iteration: 11500 Avg. Training loss: 4.1491 0.3625 sec/batch
Epoch 3/10 Iteration: 11600 Avg. Training loss: 4.1933 0.3674 sec/batch
Epoch 3/10 Iteration: 11700 Avg. Training loss: 4.1989 0.3636 sec/batch
Epoch 3/10 Iteration: 11800 Avg. Training loss: 4.1377 0.3610 sec/batch
Epoch 3/10 Iteration: 11900 Avg. Training loss: 4.1243 0.3706 sec/batch
Epoch 3/10 Iteration: 12000 Avg. Training loss: 4.1652 0.3735 sec/batch
Nearest to they: expect, worshiped, deferred, hawaiians, predated, vain, pspace, ashur,
Nearest to may: socialize, hentai, threefold, mesothelioma, featureless, endows, stops, capybaras,
Nearest to states: countries, galway, luc, owns, illich, fee, united, represent,
Nearest to his: story, languedoc, lands, bride, adultery, aspire, violation, affan,
Nearest to that: dreamlike, against, degenerate, evangelical, received, coefficient, voluntarily, subjectivity,
Nearest to a: integrals, be, sociopolitical, ribbon, ab, workingmen, applicable, within,
Nearest to with

Nearest to heavy: audacity, bullets, rinse, palmer, manny, decisive, osnabr, buoyancy,
Nearest to bbc: april, sewing, dixie, days, december, broadcasts, stamps, observances,
Nearest to operations: operation, injective, cpe, measurement, logistic, seleucids, usable, censure,
Nearest to mean: archontology, lorentz, unsolved, mpla, veszpr, enthalpy, unity, converging,
Nearest to institute: insidious, graduates, university, universities, research, amoeboids, attended, papers,
Epoch 4/10 Iteration: 15100 Avg. Training loss: 4.0089 0.3629 sec/batch
Epoch 4/10 Iteration: 15200 Avg. Training loss: 4.0044 0.3648 sec/batch
Epoch 4/10 Iteration: 15300 Avg. Training loss: 4.0169 0.3675 sec/batch
Epoch 4/10 Iteration: 15400 Avg. Training loss: 4.0725 0.3629 sec/batch
Epoch 4/10 Iteration: 15500 Avg. Training loss: 4.0635 0.3631 sec/batch
Epoch 4/10 Iteration: 15600 Avg. Training loss: 4.0592 0.3670 sec/batch
Epoch 4/10 Iteration: 15700 Avg. Training loss: 4.0802 0.3664 sec/batch
Epoch 4/10 Iteratio

...

Nearest to universe: cosmological, cosmology, mankind, subtitled, cosmologists, ophiuchus, calculable, initiatory,
Nearest to operating: missionary, unix, redefinition, exploits, setlist, minimal, compilers, linux,
Nearest to heavy: audacity, littering, disagreed, rinse, indeed, grands, maloney, nacl,
Nearest to bbc: broadcasts, december, april, observances, days, xvi, stamps, solstice,
Nearest to operations: operation, adic, holomorphic, nessie, injective, decrement, browski, sentenced,
Nearest to mean: archaeoastronomy, great, faithless, odes, swayed, mpla, converging, veszpr,
Nearest to institute: university, insidious, graduates, fukuoka, universities, medalists, technology, hofstadter,
Epoch 7/10 Iteration: 30100 Avg. Training loss: 3.9127 0.3676 sec/batch
Epoch 7/10 Iteration: 30200 Avg. Training loss: 3.9452 0.3605 sec/batch
Epoch 7/10 Iteration: 30300 Avg. Training loss: 3.9006 0.3594 sec/batch
Epoch 7/10 Iteration: 30400 Avg. Training loss: 3.9182 0.3609 sec/batch
Epoch 7/10 I

Nearest to with: and, of, the, as, in, a, for, are,
Nearest to after: in, first, return, at, cordon, subsequently, mutiny, frud,
Nearest to shows: twist, personality, golan, calamine, show, swadesh, puritan, filmed,
Nearest to universe: cosmological, cosmology, cosmologists, mankind, subtitled, bang, initiatory, accelerating,
Nearest to operating: missionary, unix, redefinition, linux, compilers, systems, exploits, setlist,
Nearest to heavy: audacity, littering, disagreed, decisive, indeed, rinse, gad, maloney,
Nearest to bbc: broadcasts, december, april, observances, days, month, xvi, solstice,
Nearest to operations: operation, holomorphic, adic, decrement, browski, cognitivism, optimality, afesd,
Nearest to mean: archaeoastronomy, great, faithless, converging, swayed, veszpr, dsm, lothal,
Nearest to institute: university, insidious, technology, gakuin, universities, graduates, hofstadter, fukuoka,
Epoch 8/10 Iteration: 34100 Avg. Training loss: 3.8530 0.3646 sec/batch
Epoch 8/10 Iter

Nearest to that: to, in, the, be, have, as, but, because,
Nearest to a: in, which, the, be, to, for, as, with,
Nearest to with: and, the, of, in, as, a, for, which,
Nearest to after: in, first, at, return, was, blaue, cordon, pennies,
Nearest to shows: personality, puritan, twist, golan, preslav, calamine, filmed, show,
Nearest to universe: cosmological, cosmology, mankind, cosmologists, bang, subtitled, azaria, cosmos,
Nearest to operating: missionary, systems, unix, linux, compilers, redefinition, os, bushnell,
Nearest to heavy: littering, audacity, indeed, disagreed, grands, rinse, decisive, clap,
Nearest to bbc: broadcasts, december, april, observances, days, chatham, solstice, month,
Nearest to operations: operation, holomorphic, decrement, adic, browski, reprisal, optimality, stare,
Nearest to mean: archaeoastronomy, faithless, wish, veszpr, enthalpy, birthdays, converging, great,
Nearest to institute: insidious, university, hofstadter, halakhah, gakuin, supervision, excerpted, g

Nearest to that: to, in, this, not, be, but, because, the,
Nearest to a: in, the, be, which, to, as, is, for,
Nearest to with: and, the, of, in, as, a, for, an,
Nearest to after: in, first, later, at, return, when, was, blaue,
Nearest to shows: show, personality, twist, puritan, calamine, medallist, bagpuss, swadesh,
Nearest to universe: cosmological, cosmology, mankind, cosmologists, subtitled, bang, zork, fictional,
Nearest to operating: missionary, unix, systems, redefinition, compilers, linux, coherence, peninsular,
Nearest to heavy: littering, rinse, indeed, grindcore, bathory, audacity, gad, disagreed,
Nearest to bbc: broadcasts, december, april, observances, days, solstice, xvi, festival,
Nearest to operations: operation, holomorphic, decrement, optimality, afesd, adic, falsity, nac,
Nearest to mean: archaeoastronomy, great, veszpr, enthalpy, wish, faithless, monkey, birthdays,
Nearest to institute: insidious, university, technology, aia, fukuoka, hofstadter, excerpted, halakhah

Nearest to a: in, the, which, is, be, to, for, an,
Nearest to with: and, in, the, of, as, a, for, are,
Nearest to after: in, first, subsequently, return, was, blaue, at, cordon,
Nearest to shows: show, filmed, twist, personality, medallist, puritan, kaos, bagpuss,
Nearest to universe: cosmology, cosmological, cosmologists, zork, bang, subtitled, cosmos, cosmic,
Nearest to operating: missionary, unix, systems, compilers, linux, redefinition, coherence, msil,
Nearest to heavy: littering, grindcore, bathory, rinse, disagreed, sennar, gad, clap,
Nearest to bbc: broadcasts, december, april, days, observances, chatham, day, festival,
Nearest to operations: operation, decrement, expedite, firms, optimality, holomorphic, nac, transclusion,
Nearest to mean: archaeoastronomy, great, mls, manu, observations, to, lothal, birthdays,
Nearest to institute: insidious, university, technology, fukuoka, aia, gakuin, excerpted, hofstadter,
Epoch 10/10 Iteration: 46100 Avg. Training loss: 3.8299 0.3582 sec

Restore the trained network if you need to:

In [ ]:
with train_graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=train_graph) as sess:
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    embed_mat = sess.run(embedding)

## Visualizing the word vectors

Below we'll use T-SNE to visualize how our high-dimensional word vectors cluster together. T-SNE is used to project these vectors into two dimensions while preserving local stucture. Check out [this post from Christopher Olah](http://colah.github.io/posts/2014-10-Visualizing-MNIST/) to learn more about T-SNE and other ways to visualize high-dimensional data.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [ ]:
viz_words = 500
tsne = TSNE()
embed_tsne = tsne.fit_transform(embed_mat[:viz_words, :])

In [ ]:
fig, ax = plt.subplots(figsize=(14, 14))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)